# Project 2 Model Classification
### Serena Shah, Osvaldo Salinas
## Part 1

### Loading the Data

In [1]:
import os
from pathlib import Path

Path("data/train/damage").mkdir(parents=True, exist_ok=True)
Path("data/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data/test/damage").mkdir(parents=True, exist_ok=True)
Path("data/test/no_damage").mkdir(parents=True, exist_ok=True)

In [2]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above

damage_all_paths = os.listdir('data_all_modified/damage')
no_damage_all_paths = os.listdir('data_all_modified/no_damage')

In [3]:
# split the image paths into train and test by randomly selecting 80% of the images in train and 20% in test.
import random

print("...............")
train_damage_paths = random.sample(damage_all_paths, int(len(damage_all_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in damage_all_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))
print("...............\n")
print("...............")
train_no_damage_paths = random.sample(no_damage_all_paths, int(len(no_damage_all_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in no_damage_all_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))
print("...............\n")

...............
train damage image count:  11336
test damage image count:  2834
len of overlap:  0
...............

...............
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0
...............



In [4]:
# copying of files in the train and test directories
import shutil

root_dir = 'data_all_modified'
split_root_dir = 'data'

# Copy damaged images to train and test directories
for p in train_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'damage', p), os.path.join(split_root_dir, 'train/damage', p))

for p in test_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'damage', p), os.path.join(split_root_dir, 'test/damage', p))

# Copy no damage images to train and test directories
for p in train_no_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'no_damage', p), os.path.join(split_root_dir, 'train/no_damage', p))

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join(root_dir, 'no_damage', p), os.path.join(split_root_dir, 'test/no_damage', p))

# Check counts to ensure files are copied correctly
print("Files in train/damage: ", len(os.listdir(os.path.join(split_root_dir, "train/damage"))))
print("Files in train/no_damage: ", len(os.listdir(os.path.join(split_root_dir, "train/no_damage"))))
print("Files in test/damage: ", len(os.listdir(os.path.join(split_root_dir, "test/damage"))))
print("Files in test/no_damage: ", len(os.listdir(os.path.join(split_root_dir, "test/no_damage"))))

Files in train/damage:  14062
Files in train/no_damage:  7102
Files in test/damage:  6924
Files in test/no_damage:  3506


### Data preprocessing

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Rescaling
train_data_dir = 'data/train/'

batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

2024-04-11 00:16:23.122876: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 00:16:23.193370: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 00:16:23.576842: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 00:16:23.576921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 00:16:23.644887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Found 21164 files belonging to 2 classes.
Using 16932 files for training.
Using 4232 files for validation.


In [65]:
test_data_dir = 'data/test/'

batch_size = 1
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 10430 files belonging to 2 classes.


## Part 2
### ANN

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model_ann = Sequential()

# Flatten
model_ann.add(Flatten(input_shape=(img_height, img_width, 3)))

# Our input layer can have any number of perceptrons, we chose 2, however,
# the input dimension must match the number of features in the independent variable -- therefore, we set
# it to 4
model_ann.add(Dense(120, input_shape = (img_height*img_width,), activation='relu'))

# we can add any number of hidden layers with any number of perceptrons; here we choose 1 layer with 128 perceptrons. The
# hidden layers should all use RELU
model_ann.add(Dense(128, activation='relu'))

# softmax activation function is selected for multi-label classification problems; there are 3 perceptrons in this
# last layer because there are 2 target labels to predict (it matches the shape of y)
model_ann.add(Dense(1, activation='sigmoid'))

# compile model
model_ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# generating summary of model
model_ann.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_14 (Flatten)        (None, 49152)             0         
                                                                 
 dense_42 (Dense)            (None, 120)               5898360   
                                                                 
 dense_43 (Dense)            (None, 128)               15488     
                                                                 
 dense_44 (Dense)            (None, 1)                 129       
                                                                 
Total params: 5913977 (22.56 MB)
Trainable params: 5913977 (22.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
#fit the model from image generator
history_ann = model_ann.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20


/usr/local/lib/python3.11/site-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


530/530 [==============================] - 44s 82ms/step - loss: 0.0000e+00 - accuracy: 0.6678 - val_loss: 0.0000e+00 - val_accuracy: 0.6486
Epoch 2/20
530/530 [==============================] - 38s 72ms/step - loss: 0.0000e+00 - accuracy: 0.6684 - val_loss: 0.0000e+00 - val_accuracy: 0.6486
Epoch 3/20
530/530 [==============================] - 35s 65ms/step - loss: 0.0000e+00 - accuracy: 0.6684 - val_loss: 0.0000e+00 - val_accuracy: 0.6486
Epoch 4/20
530/530 [==============================] - 32s 59ms/step - loss: 0.0000e+00 - accuracy: 0.6684 - val_loss: 0.0000e+00 - val_accuracy: 0.6486
Epoch 5/20
530/530 [==============================] - 37s 69ms/step - loss: 0.0000e+00 - accuracy: 0.6684 - val_loss: 0.0000e+00 - val_accuracy: 0.6486
Epoch 6/20
530/530 [==============================] - 37s 70ms/step - loss: 0.0000e+00 - accuracy: 0.6684 - val_loss: 0.0000e+00 - val_accuracy: 0.6486
Epoch 7/20
530/530 [==============================] - 36s 67ms/step - loss: 0.0000e+00 - accuracy: 

In [25]:
test_loss_ann, test_accuracy_ann = model_ann.evaluate(test_rescale_ds, verbose=0)
test_accuracy_ann

0.6638542413711548

In [26]:
model_ann.save("models/ann.keras")

### LeNet-5

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import RMSprop
import pandas as pd

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(3, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d_12 (Aver  (None, 63, 63, 6)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_25 (Conv2D)          (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_13 (Aver  (None, 30, 30, 16)        0         
 agePooling2D)                                                   
                                                                 
 flatten_15 (Flatten)        (None, 14400)             0         
                                                                 
 dense_45 (Dense)            (None, 120)             

In [28]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
530/530 [==============================] - 23s 42ms/step - loss: 0.6043 - accuracy: 0.6919 - val_loss: 0.7452 - val_accuracy: 0.6498
Epoch 2/20
530/530 [==============================] - 29s 54ms/step - loss: 0.5020 - accuracy: 0.7684 - val_loss: 0.6439 - val_accuracy: 0.6812
Epoch 3/20
530/530 [==============================] - 24s 46ms/step - loss: 0.4373 - accuracy: 0.8157 - val_loss: 0.4048 - val_accuracy: 0.8296
Epoch 4/20
530/530 [==============================] - 33s 61ms/step - loss: 0.4026 - accuracy: 0.8375 - val_loss: 0.4826 - val_accuracy: 0.7833
Epoch 5/20
530/530 [==============================] - 27s 50ms/step - loss: 0.3819 - accuracy: 0.8514 - val_loss: 0.5876 - val_accuracy: 0.7472
Epoch 6/20
530/530 [==============================] - 30s 56ms/step - loss: 0.3614 - accuracy: 0.8596 - val_loss: 0.3471 - val_accuracy: 0.8575
Epoch 7/20
530/530 [==============================] - 28s 52ms/step - loss: 0.3412 - accuracy: 0.8666 - val_loss: 0.3207 - val_accuracy:

The Lenet-5 model accuracy is shown below.

In [29]:
test_loss_lenet5, test_accuracy_lenet5 = model_lenet5.evaluate(test_rescale_ds, verbose=0)
test_accuracy_lenet5

0.9396932125091553

In [30]:
model_lenet5.save("models/lenet5.keras")

### Alt LeNet-5

In [69]:
from keras.optimizers import RMSprop
import pandas as pd

model_altlenet = models.Sequential()

# Layer 1: Convolutional layer with 32 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 64 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional layer with 128 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Convolutional layer with 128 filters of size 3x3, followed by max pooling
model_altlenet.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_altlenet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_altlenet.add(layers.Flatten())

# Adding dropout prevents overfitting
model_altlenet.add(layers.Dropout(0.2))

# Layer 5: Fully connected layer with 120 neurons
model_altlenet.add(layers.Dense(120, activation='relu'))

# Layer 6: Fully connected layer with 84 neurons
model_altlenet.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_altlenet.add(layers.Dense(3, activation='softmax'))

# Compile model
model_altlenet.compile(optimizer=RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_altlenet.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_35 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_36 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 14, 14, 128)     

In [70]:
#fit the model from image generator
history_altlenet = model_altlenet.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
530/530 [==============================] - 127s 238ms/step - loss: 0.5727 - accuracy: 0.7210 - val_loss: 0.7622 - val_accuracy: 0.6805
Epoch 2/20
530/530 [==============================] - 123s 231ms/step - loss: 0.4021 - accuracy: 0.8263 - val_loss: 0.4625 - val_accuracy: 0.8041
Epoch 3/20
530/530 [==============================] - 118s 222ms/step - loss: 0.3221 - accuracy: 0.8662 - val_loss: 1.1519 - val_accuracy: 0.6902
Epoch 4/20
530/530 [==============================] - 124s 233ms/step - loss: 0.2438 - accuracy: 0.9042 - val_loss: 0.1946 - val_accuracy: 0.9272
Epoch 5/20
530/530 [==============================] - 141s 265ms/step - loss: 0.1847 - accuracy: 0.9275 - val_loss: 0.1418 - val_accuracy: 0.9433
Epoch 6/20
530/530 [==============================] - 125s 235ms/step - loss: 0.1487 - accuracy: 0.9406 - val_loss: 0.1452 - val_accuracy: 0.9449
Epoch 7/20
530/530 [==============================] - 135s 254ms/step - loss: 0.1318 - accuracy: 0.9481 - val_loss: 0.1155 -

In [33]:
test_loss_altlenet, test_accuracy_altlenet = model_altlenet.evaluate(test_rescale_ds, verbose=0)
test_accuracy_altlenet

0.9832214713096619

In [71]:
model_lenet5.save("models/altlenet.keras")

## Part 3

In [63]:
import requests

In [86]:
import numpy as np
# grab an entry from  -- here, we grab the first one
iterator = iter(test_rescale_ds)

l = iterator.get_next()[0].numpy().tolist()

In [82]:
# make the POST request passing the single test case as the `image` field:
rsp = requests.post("http://172.17.0.1:5000/models", json={"image": l})

# print the json response
rsp.json()

{'result': [[0.365214079618454, 0.6347780227661133, 7.855040166759863e-06],
  [0.06608804315328598, 0.9339077472686768, 4.163519406574778e-06],
  [0.09341005235910416, 0.9065787196159363, 1.1243201697652694e-05],
  [0.8905065059661865, 0.10949257761240005, 8.964684639067855e-07],
  [0.0009334941860288382, 0.9990662932395935, 1.7873585989036656e-07],
  [0.980830729007721, 0.01916562207043171, 3.576124981918838e-06],
  [0.9869367480278015, 0.013055889867246151, 7.445906703651417e-06],
  [0.9980077147483826, 0.0019904552027583122, 1.916544533742126e-06],
  [0.9340019822120667, 0.06599751859903336, 5.926378321419179e-07],
  [0.1867174357175827, 0.8132746815681458, 7.863062819524202e-06],
  [0.4082598090171814, 0.5917390584945679, 1.1727588571375236e-06],
  [0.8756382465362549, 0.12428555637598038, 7.616321818204597e-05],
  [0.8489068746566772, 0.15100687742233276, 8.627916395198554e-05],
  [0.006948060356080532, 0.993046224117279, 5.730778411816573e-06],
  [0.008656494319438934, 0.99134117